E1
T1.1
ProductName:    	macOS
ProductVersion:    	15.0.1
BuildVersion:    	24A348
Darwin Mac 24.0.0 Darwin Kernel Version 24.0.0: Tue Sep 24 23:37:13 PDT 2024; root:xnu-11215.1.12~1/RELEASE_ARM64_T8112 arm64

time.time: 7.152557373046875e-07
timeit.default_timer: 8.297502063214779e-08
time.time_ns: 768.0


In [1]:
import numpy as np
import time
import timeit


def checktick(timestampFunction):
    M = 200
    timesfound = np.empty((M,))
    for i in range(M):
        t1 = timestampFunction()  # get timestamp from timer
        t2 = timestampFunction()  # get timestamp from timer
        while (
            t2 - t1
        ) < 1e-16:  # if zero then we are below clock granularity, retake timing
            t2 = timestampFunction()  # get timestamp from timer
        t1 = t2  # this is outside the loop
        timesfound[i] = t1  # record the time stamp
    minDelta = 1000000
    Delta = np.diff(timesfound)  # it should be cast to int only when needed
    minDelta = Delta.min()
    return minDelta


def main():
    print(checktick(time.time))
    print(checktick(timeit.default_timer))
    print(checktick(time.time_ns))


if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'numpy'

With the JuliaSet.py using our own made decorator.py

Results for computer with core speed of 4.05 GHz (assumed):
Function: calculate_z_serial_purepython
  Average Execution Time: 2.619 seconds
  Average Standard Deviation: 0.0103 seconds
Function: calc_pure_python
  Average Execution Time: 2.811 seconds
  Average Standard Deviation: 0.0215 seconds

With a clock frequency of about 4.05 GHz, a single cycle is about 0.25ns which is vastly smaller than both standard deviations (as they have 10.3 ms and 21.5 ms respectively). This means the amount of cycles must differ between each run or other programmes take up cycles on the system when running the code.

It might as well be the case that the OS decides to not use the performance cores in the CPU. These have a lower clock frequency of 2.75 GHz which results in the per cycle time of 0.36ns. This is still magnitudes smaller than the standard deviation of 10.3 ms and 21.5 ms of the average standard deviations. The kernel thread scheduling could allocate varying amounts of CPU time to each thread, contributing to the observed variation in execution times.


In [ ]:
import timeit
from functools import wraps
from typing import Callable
import numpy as np


def timer(func: Callable) -> Callable:
    @wraps(func)
    def wrapper(*args, **kwargs):
        arr = np.zeros((10,))
        result = None
        for n in range(10):
            start = timeit.default_timer()
            result = func(*args, **kwargs)
            end = timeit.default_timer()
            arr[n] = end - start
        print(f"Function: {func.__name__}")
        print(f"Average Execution Time: {arr.mean():.6f} seconds")
        print(f"Standard Deviation: {arr.std():.6f} seconds")
        return result

    return wrapper
